Springboard Grader:

Not sure that it applies here, but the phpmyadmin site was not available for large amounts of time during my completing this.


In [9]:
import sqlite3
import pandas as pd

db = sqlite3.connect('/Users/jshensley/Desktop/springboard/DS_Method_and_Capstones/SQL2/sqlite_db_pythonsqlite.db')
cursor = db.cursor()

In [10]:
facilities = ['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintencance']

In [11]:
# test the connection and customized output
a = cursor.execute("SELECT * FROM Facilities")
adf = pd.DataFrame(a, columns=facilities)
adf

,facid,name,membercost,guestcost,initialoutlay,monthlymaintencance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


Q10: Produce a list of facilities with a total revenue less than 1000.
        The output of facility name and total revenue, sorted by revenue.
        Remember that there's a different cost for guests and members!

In [12]:
revenue_lt_1000 = cursor.execute(
        "WITH rev AS "
        "(SELECT Facilities.name, "
        "CASE WHEN Bookings.memid = 0 "
        "THEN Facilities.guestcost * Bookings.slots "
        "ELSE Facilities.membercost * Bookings.slots END AS revenue FROM Facilities INNER JOIN Bookings ON Facilities.facid = Bookings.facid)"

        "SELECT rev.name AS name, SUM(revenue) AS totals FROM rev "
        "GROUP By name HAVING totals < 1000"
        )

revenue_lt_1000_df = pd.DataFrame(revenue_lt_1000, columns=['name', 'totals'])
revenue_lt_1000_df

,name,totals
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

note: The fetchall() call to a variable was due to pandas not liking the pipe characters as discovered on as opposed to the previous cell where I was able to call the df variable directly.

In [13]:
referrals = cursor.execute("SELECT (m1.surname ||' '|| m1.firstname) AS membername, (m2.surname ||' '|| m2.firstname) AS referredby FROM Members as m1 "
                           "INNER JOIN Members as m2 ON m1.recommendedby = m2.memid")
refer = referrals.fetchall()
refers = pd.DataFrame(refer, columns=['member_name', 'referred_by'])
refers

,member_name,referred_by
0,Joplette Janice,Smith Darren
1,Butters Gerald,Smith Darren
2,Dare Nancy,Joplette Janice
3,Boothe Tim,Rownam Tim
4,Stibbons Ponder,Tracy Burton
5,Owen Charles,Smith Darren
6,Jones David,Joplette Janice
7,Baker Anne,Stibbons Ponder
8,Smith Jack,Smith Darren
9,Bader Florence,Stibbons Ponder


Q12: Find the facilities with their usage by member, but not guests

In [14]:
member_usage = cursor.execute("WITH users AS (SELECT facid, memid FROM Bookings WHERE memid != 0)"
                              "SELECT Facilities.name AS fac, COUNT(users.memid) AS count FROM Facilities LEFT JOIN users ON Facilities.facid = users.facid GROUP BY facilities.name")

fetchall_members = member_usage.fetchall()

facilities_df = pd.DataFrame(fetchall_members, columns=['facility', 'usage'])
facilities_df

,facility,usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


Q13: Find the facilities usage by month, but not guests

In [15]:
facilities_usage = cursor.execute("WITH usage AS (SELECT memid, strftime('%m', starttime) as month FROM Bookings WHERE memid != 0) SELECT COUNT(usage.memid) AS cnt, usage.month FROM usage GROUP BY usage.month")

fetchall_usage = facilities_usage.fetchall()

usage_df = pd.DataFrame(fetchall_usage, columns=['usage', 'month'])
usage_df

,usage,month
0,480,07
1,1168,08
2,1512,09


In [16]:
cursor.close()
db.close()